In [9]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
from numba import jit

import astropy.units as u
from astropy import wcs
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.visualization import make_lupton_rgb
from astropy.utils.data import download_file, clear_download_cache

import matplotlib.pyplot as plt
from matplotlib import rcParams

print(np.__version__)

rcParams.update({'axes.linewidth': 1.5})
rcParams.update({'xtick.direction': 'in'})
rcParams.update({'ytick.direction': 'in'})
rcParams.update({'xtick.minor.visible': 'True'})
rcParams.update({'ytick.minor.visible': 'True'})
rcParams.update({'xtick.major.pad': '7.0'})
rcParams.update({'xtick.major.size': '8.0'})
rcParams.update({'xtick.major.width': '1.5'})
rcParams.update({'xtick.minor.pad': '7.0'})
rcParams.update({'xtick.minor.size': '4.0'})
rcParams.update({'xtick.minor.width': '1.5'})
rcParams.update({'ytick.major.pad': '7.0'})
rcParams.update({'ytick.major.size': '8.0'})
rcParams.update({'ytick.major.width': '1.5'})
rcParams.update({'ytick.minor.pad': '7.0'})
rcParams.update({'ytick.minor.size': '4.0'})
rcParams.update({'ytick.minor.width': '1.5'})
rcParams.update({'axes.titlepad': '10.0'})
rcParams.update({'font.size': 25})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.16.4


In [70]:
@jit(nopython=True, nogil=True)
def cart2polar(x, y):
    """
    Transform Cartesian coordinates to polar
    
    Parameters
    ----------
    x, y : floats or arrays
        Cartesian coordinates
    
    Returns
    -------
    r, theta : floats or arrays
        Polar coordinates
        
    """
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(x, y)  # θ referenced to vertical
    return r, theta

@jit(nopython=True, nogil=True)
def polar2cart(r, theta):
    """
    Transform polar coordinates to Cartesian
    
    Parameters
    -------
    r, theta : floats or arrays
        Polar coordinates
        
    Returns
    ----------
    x, y : floats or arrays
        Cartesian coordinates
    """
    y = r * np.cos(theta)   # θ referenced to vertical
    x = r * np.sin(theta)
    return x, y

def index_coords(data):
    """
    Creates x & y coords for the indicies in a numpy array
    
    Parameters
    ----------
    data : numpy array
        2D data
    origin : (x,y) tuple
        defaults to the center of the image. Specify origin=(0,0)
        to set the origin to the *bottom-left* corner of the image.
    
    Returns
    -------
        x, y : arrays
    """
    ny, nx = data.shape[:2]
    
    x, y = np.meshgrid(
        np.arange(float(nx)), np.arange(float(ny)))

    return x, y

@jit(nopython=True, nogil=True)
def index_to_galcen(x_idx, y_idx, x_cen, y_cen):
    return x_idx - x_cen, y_idx - y_cen

In [54]:
x, y = np.arange(100) * 1.0, np.arange(100) * 1.0

data = np.zeros([1000, 1000])

r, theta = cart2polar(x, y)
xx, yy = polar2cart(r, theta)

xid, yid = index_coords(data)

In [13]:
%timeit r, thetha = cart2polar(x, y)

2.05 µs ± 6.46 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
%timeit xx, yy = polar2cart(r, theta)

3.77 µs ± 33.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [55]:
%timeit xid, yid = index_coords(data)

1.6 ms ± 38.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [69]:
%timeit x0, y0 = index_to_galcen(xid, yid, 500.0, 500.0)

2.21 ms ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


range(0, 1000)